In [25]:
import re
import json
import requests
import time
from random import uniform

def get_shopee_ratings(url):
    # Extract shop_id and item_id from URL
    r = re.search(r'i\.(\d+)\.(\d+)', url)
    if not r:
        print("Invalid URL format")
        return None
    
    shop_id, item_id = r.group(1), r.group(2)
    
    # Create session object
    session = requests.Session()
    
    # Updated headers with SPC_EC cookie and csrf token
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Referer': url,
        'Origin': 'https://shopee.com.my',
        'Connection': 'keep-alive',
        'sec-ch-ua': '"Google Chrome";v="119"',
        'sec-ch-ua-platform': '"Windows"',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
    }
    
    try:
        # First, get the necessary cookies
        print("Getting initial cookies...")
        initial_response = session.get(
            'https://shopee.com.my',
            headers=headers
        )
        time.sleep(2)
        
        # Get the product page to obtain additional tokens
        print("Visiting product page...")
        product_response = session.get(url, headers=headers)
        time.sleep(2)
        
        # Updated API endpoint
        api_url = (
            f'https://shopee.com.my/api/v4/item/get_ratings'
            f'?filter=0'
            f'&flag=1'
            f'&itemid={item_id}'
            f'&limit=6'
            f'&offset=0'
            f'&shopid={shop_id}'
            f'&type=0'
        )
        
        print("Fetching ratings...")
        response = session.get(api_url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            print("Successfully retrieved data!")
            print("\nFirst few ratings:")
            for rating in data.get('data', {}).get('ratings', [])[:3]:
                print(f"User: {rating.get('author_username', 'Unknown')}")
                print(f"Rating: {rating.get('rating_star', 'N/A')}")
                print(f"Comment: {rating.get('comment', 'No comment')}")
                print('-' * 80)
        else:
            print(f"Error: {response.status_code}")
            print("Response:", response.text)
            
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
    finally:
        session.close()

# Usage
if __name__ == "__main__":
    url = 'https://shopee.com.my/Ins-Style-Niche-Exquisite-Flower-Graffiti-Cartoon-for-iPhone-Case-15-14-13-12-11-pro-max-7-8-plus-xr-x-xs-max-15pro-i.836359116.24714250751'
    get_shopee_ratings(url)

Getting initial cookies...
Visiting product page...
Fetching ratings...
Error: 403
Response: {"is_customized":false,"is_login":false,"action_type":2,"error":90309999,"tracking_id":"63745ae9672-b99e-47eb-bed6-70276a5c7921"}

